In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,RepeatedKFold,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV,Lasso,LassoCV,ElasticNet,ElasticNetCV,LogisticRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeClassifier,plot_tree,export_text
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor,ExtraTreesClassifier,ExtraTreesRegressor,BaggingClassifier
from sklearn.ensemble import BaggingRegressor,AdaBoostClassifier,AdaBoostRegressor,GradientBoostingClassifier,GradientBoostingRegressor
from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBClassifier,XGBRegressor
from lightgbm import LGBMClassifier,LGBMRegressor
from sklearn.svm import SVC,SVR,LinearSVC,LinearSVR
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.cluster import OPTICS,Birch,SpectralClustering,KMeans,MeanShift,MiniBatchKMeans,DBSCAN,AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import make_classification

In [35]:
df=sns.load_dataset('tips')

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [9]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [11]:
df.shape

(244, 7)

In [13]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [15]:
display(df['sex'].value_counts())
display(df['smoker'].value_counts())
display(df['day'].value_counts())
display(df['time'].value_counts())

sex
Male      157
Female     87
Name: count, dtype: int64

smoker
No     151
Yes     93
Name: count, dtype: int64

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [17]:
#the columns such as ['sex','smoker','time'] are having binary classes so use LabelEncoding and the column 'day' is having multiple category so #use OneHotEnding
#Before encoding itself we have to split train and test data to avoid dataleakage thatis the model that is build shouldnt know anything abouyt the test data.
#df[['sex','smoker','time']]=LabelEncoder().fit_transform(df[['sex','smoker','time']])
df.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [36]:
y=df['total_bill']
X=df.drop('total_bill',axis=1)

In [37]:
X.columns

Index(['tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [23]:
y

0      16.99
1      10.34
2      21.01
3      23.68
4      24.59
       ...  
239    29.03
240    27.18
241    22.67
242    17.82
243    18.78
Name: total_bill, Length: 244, dtype: float64

In [38]:
Xtr,Xte,ytr,yte=train_test_split(X,y,test_size=0.25,random_state=42)
print(Xtr.shape)
print(Xte.shape)
print(ytr.shape)
print(yte.shape)

(183, 6)
(61, 6)
(183,)
(61,)


In [39]:

col=['sex','smoker','time']
for i in col:
    le=LabelEncoder()
    Xtr[i]=le.fit_transform(Xtr[i])
    Xte[i]=le.transform(Xte[i])
display(Xtr.head(2))
display(Xte.head(2))
    

,tip,sex,smoker,day,time,size
115,3.50,0,0,Sun,0,2
181,5.65,1,1,Sun,0,2


,tip,sex,smoker,day,time,size
24,3.18,1,0,Sat,0,2
6,2.00,1,0,Sun,0,2


In [40]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(drop='first'), [3])],
    remainder='passthrough'
)

In [41]:
print(Xte.shape)
print(Xtr.shape)

(61, 6)
(183, 6)


In [42]:
import sys
np.set_printoptions(threshold=sys.maxsize)
Xtr=ct.fit_transform(Xtr)

In [43]:
Xte=ct.transform(Xte)

In [44]:
print(Xte.shape)
print(Xtr.shape)

(61, 8)
(183, 8)


In [45]:
print(Xte.dtype)
print(Xtr.dtype)

float64
float64


In [46]:

from sklearn.metrics import mean_squared_error,r2_score
sv=SVR()
sv.fit(Xtr,ytr)
ypred=sv.predict(Xte)
mse=mean_squared_error(yte,ypred)
print(f"the error is {mse}")
print(f"the root mse is {np.sqrt(mse)}")
print(f" the rsquared score is {r2_score(yte,ypred)}")

the error is 39.31122612339172
the root mse is 6.269866515595983
 the rsquared score is 0.49798620106004743


In [47]:
params={'C':[1,0.1,10,100],
        'gamma':[0.1,1,0.01,0.001],
        'kernel':['rbf']}
gsv=GridSearchCV(SVR(),param_grid=params,refit=True,cv=5)
gsv.fit(Xtr,ytr)


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 0.1, 10, 100], 'gamma': [0.1, 1, 0.01, 0.001],
                         'kernel': ['rbf']})

In [48]:
gsv.best_params_

{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}

In [51]:
sv1=SVR(C= 100, gamma= 0.01, kernel= 'rbf')
sv1.fit(Xtr,ytr)
ypred=sv1.predict(Xte)
mse=mean_squared_error(yte,ypred)
print(f"the error is {mse}")
print(f"the root mse is {np.sqrt(mse)}")
print(f" the rsquared score is {r2_score(yte,ypred)}")

the error is 34.21448856486327
the root mse is 5.8493152218754005
 the rsquared score is 0.5630727637616448
